In [1]:
%load_ext autoreload
%autoreload 2

In [7]:
import json

import llama_cpp
import torch

from curverag import utils
from curverag.curverag import CurveRAG, DEFAULT_ENTITY_TYPES, DEFAULT_GLINER_MODEL, DEFAULT_SENTENCE_TRANSFORMER_MODEL
from curverag.graph import KnowledgeGraph
from curverag.atth.kg_dataset import KGDataset
from curverag.atth.models.hyperbolic import AttH
from curverag.eval import evaluation, queries, context, expected_output

In [ ]:
max_tokens = 10000
n_ctx=10000

llm, outlines_model = utils.load_model(
    llm_model_path="./models/Meta-Llama-3-8B-Instruct.Q6_K.gguf",
    tokenizer=llama_cpp.llama_tokenizer.LlamaHFTokenizer.from_pretrained("unsloth/Llama-3.2-1B-Instruct"),
    n_ctx=n_ctx,
    max_tokens=max_tokens
)

In [25]:
entity_types = ['person', 'location', 'name']
rag = CurveRAG(
    llm,
    outlines_model,
    entity_types,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

In [18]:
docs

['\nJack and Jill are adult siblings. Jakc just turnt 45 years old as it was his birthday 2 days ago and Jill is also 45; you gussed it they are twins. Jack and Jill bumped into their friend Sam and Sam seemed to be very upset. They know Sam has been single for a long time so they told Sam to talk to their friend who they think could be a good match for Sam. One year later and Sam and hey friend were dating and had even moved to Romania in a massive Castle. Sam is very happy now and they have a "happily ever after" story, which is fantastic.\n']

In [22]:
docs = [
    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",
    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",
    "Administer 5mg of lorazepam intravenously for acute seizure management.",
    "Blood tests showed elevated ALT and AST levels, indicating possible liver inflammation.",
    "The subject reported chronic lower back pain, managed with physical therapy and NSAIDs.",
    "CT angiography confirmed the presence of a pulmonary embolism in the right lower lobe.",
    "The patient underwent coronary artery bypass graft surgery without complications.",
    "Routine vaccination included MMR, tetanus, and influenza immunizations.",
    "Histopathology indicated ductal carcinoma in situ (DCIS) in the breast biopsy sample.",
    "The child presented with a persistent cough and fever, diagnosed as streptococcal pharyngitis."
]

In [32]:
docs = [
    context,
    "The patient was diagnosed with type 2 diabetes mellitus and prescribed metformin 500mg twice daily.",
    "MRI scan revealed a small lesion in the left temporal lobe suggestive of low-grade glioma.",
]

In [33]:
rag.fit(docs, dataset_name='jack_jill')

creating graph


  0%|                                                                                                                                                                                           | 0/3 [00:00<?, ?it/s]llama_perf_context_print:        load time =   17670.87 ms
llama_perf_context_print: prompt eval time =   18923.90 ms /   721 tokens (   26.25 ms per token,    38.10 tokens per second)
llama_perf_context_print:        eval time =  697234.84 ms /   466 runs   ( 1496.21 ms per token,     0.67 tokens per second)
llama_perf_context_print:       total time =  720554.57 ms /  1187 tokens
 33%|███████████████████████████████████████████████████████████▎                                                                                                                      | 1/3 [12:00<24:01, 720.58s/it]llama_perf_context_print:        load time =   17670.87 ms
llama_perf_context_print: prompt eval time =   11584.95 ms /   959 tokens (   12.08 ms per token,    82.78 tokens per second)
ll

creating dataset
train kg embeddings


train loss: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 94.38ex/s, loss=0.6883]
2025-05-21 01:05:58,827 INFO     	 Epoch 0 | average train loss: 0.6883
2025-05-21 01:05:58,827 INFO     	 Epoch 0 | average train loss: 0.6883
2025-05-21 01:05:58,827 INFO     	 Epoch 0 | average train loss: 0.6883
2025-05-21 01:05:58,830 INFO     	 Epoch 0 | average valid loss: 2.1058
2025-05-21 01:05:58,830 INFO     	 Epoch 0 | average valid loss: 2.1058
2025-05-21 01:05:58,830 INFO     	 Epoch 0 | average valid loss: 2.1058
train loss: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 129.59ex/s, loss=2.1058]
2025-05-21 01:05:59,064 INFO     	 Epoch 1 | average train loss: 2.1058
2025-05-21 01:05:59,064 INFO     	 Epoch 1 | average train loss: 2

rhs
got (6, 3)
lhs
got (11, 3)


...

train loss: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 220.61ex/s, loss=0.0316]
2025-05-21 01:06:05,237 INFO     	 Epoch 45 | average train loss: 0.0316
2025-05-21 01:06:05,237 INFO     	 Epoch 45 | average train loss: 0.0316
2025-05-21 01:06:05,237 INFO     	 Epoch 45 | average train loss: 0.0316
2025-05-21 01:06:05,241 INFO     	 Epoch 45 | average valid loss: 1.0451
2025-05-21 01:06:05,241 INFO     	 Epoch 45 | average valid loss: 1.0451
2025-05-21 01:06:05,241 INFO     	 Epoch 45 | average valid loss: 1.0451
train loss: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 244.63ex/s, loss=0.0376]
2025-05-21 01:06:05,365 INFO     	 Epoch 46 | average train loss: 0.0376
2025-05-21 01:06:05,365 INFO     	 Epoch 46 | average train

rhs
got (6, 3)
lhs
got (11, 3)


train loss: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 262.49ex/s, loss=0.0216]
2025-05-21 01:06:05,605 INFO     	 Epoch 48 | average train loss: 0.0216
2025-05-21 01:06:05,605 INFO     	 Epoch 48 | average train loss: 0.0216
2025-05-21 01:06:05,605 INFO     	 Epoch 48 | average train loss: 0.0216
2025-05-21 01:06:05,608 INFO     	 Epoch 48 | average valid loss: 0.9730
2025-05-21 01:06:05,608 INFO     	 Epoch 48 | average valid loss: 0.9730
2025-05-21 01:06:05,608 INFO     	 Epoch 48 | average valid loss: 0.9730
train loss: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 30/30 [00:00<00:00, 268.80ex/s, loss=0.0190]
2025-05-21 01:06:05,722 INFO     	 Epoch 49 | average train loss: 0.0190
2025-05-21 01:06:05,722 INFO     	 Epoch 49 | average train

rhs
got (6, 3)
lhs
got (11, 3)
rhs
got (1, 4)
got (10, 2)
got (2, 12)
lhs
got (2, 4)
got (8, 2)
got (0, 12)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [35]:
rag.graph.nodes

[Node(id=1, name='Jack', description='Jack is an adult sibling', alias=['Jack']),
 Node(id=2, name='Jill', description='Jill is an adult sibling', alias=['Jill']),
 Node(id=3, name='Sam', description='Sam is a friend of Jack and Jill', alias=['Sam']),
 Node(id=4, name='Romania', description='A country in Eastern Europe', alias=['Romania']),
 Node(id=5, name='Castle', description='A large building', alias=['Castle']),
 Node(id=6, name="Jack's friend", description='A friend of Jack and Jill who is a potential match for Sam', alias=["Jack's friend"]),
 Node(id=7, name='Patient', description='A person who received a medical diagnosis and treatment', alias=['Patient']),
 Node(id=8, name='Type 2 diabetes mellitus', description='A type of diabetes', alias=['Type 2 diabetes', 'Diabetes mellitus']),
 Node(id=9, name='Metformin', description='A medication used to treat type 2 diabetes', alias=['Metformin 500mg']),
 Node(id=10, name='Diagnosis', description='The process of identifying a medical c

In [38]:
resp = rag.query(queries[0])
resp

entities ['Jack']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

[[np.int64(0)]]
node_ids [tensor([8, 6])]
node_ids [8, 6]

You are a helpful assistant analyzing the given input data to provide an helpful response to the user query.

# USER QUERY
How old is Jack?

# Context:
KnowledgeGraph Overview
  There are 3 entities and 0 relationships in this graph.

Entities in the graph:
  • 'Jack':
      The entity has the following description: Jack is an adult sibling
      It can also be referred to as: Jack
  • 'Jack's friend':
      The entity has the following description: A friend of Jack and Jill who is a potential match for Sam
      It can also be referred to as: Jack's friend
  • 'Type 2 diabetes mellitus':
      The entity has the following description: A type of diabetes
      It can also be referred to as: Type 2 diabetes, Diabetes mellitus

Relationships between nodes:



# INSTRUCTIONS
Your goal is to provide a response to the user query using the relevant information in the input data:
- the "Entities" and "Relationships" tables contain hig

Llama.generate: 470 prefix-match hit, remaining 1 prompt tokens to eval
llama_perf_context_print:        load time =   17670.87 ms
llama_perf_context_print: prompt eval time =  457403.48 ms /     2 tokens (228701.74 ms per token,     0.00 tokens per second)
llama_perf_context_print:        eval time =  638662.94 ms /    99 runs   ( 6451.14 ms per token,     0.16 tokens per second)
llama_perf_context_print:       total time =  645344.16 ms /   101 tokens


'The query is "How old is Jack?"\n\nThe user query is asking for the age of a person named Jack. \nAccording to the "Entities" table, Jack is an adult sibling. \nThere is no information provided about Jack\'s age.\n\n**Response:** I couldn\'t find any information about Jack\'s age [0]. If you know more context about Jack, please feel free to provide it, and I\'ll do my best to help you. [0] (Source: None)  #'